In [59]:
today_date = '9999-09-09'

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 7, Finished, Available, Finished)

In [60]:
from pyspark.sql.functions import *

fabric_bronze_path = f"abfss://New_Project@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/bronze_data"

df = spark.read.format('delta').load(fabric_bronze_path).filter(col('processing_date')== str(today_date))

display(df)

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7a971c1b-792e-4a6f-a314-6ceab08ce1b0)

In [ ]:
display(df)

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 393705d1-8843-46a8-90d3-bbf244e41aa1)

### Handiling Duplicates


In [61]:
print('count of rows before deleting dupicates:', df.count())
df_nodups = df.dropDuplicates()
print('count of rows before deleting duplictaes:',df_nodups.count())

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 10, Finished, Available, Finished)

count of rows before deleting dupicates: 8


count of rows before deleting duplictaes: 8


### Handle Missing or Null values


In [62]:
df_dropped = df_nodups.dropna(subset = ['Student_ID','Course_ID','Enrollment_Date'])
print('count of rows after droppping the na rows:',df_dropped.count())

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 11, Finished, Available, Finished)

count of rows after droppping the na rows: 8


In [63]:
df_filled = df_dropped.fillna({
                "Age": 0,
                "Gender": "Unknown",
                "Status": "In-progress",
                "Final_Grade": "N/A",
                "Attendance_Rate": 0.0,
                "Time_Spent_on_Course_hrs": 0.0,
                "Assignments_Completed": 0,
                "Quizzes_Completed": 0,
                "Forum_Posts": 0,
                "Messages_Sent": 0,
                "Quiz_Average_Score": 0.0,
                "Assignment_Average_Score": 0.0,
                "Project_Score": 0.0,
                "Extra_Credit": 0.0,
                "Overall_Performance": 0.0,
                "Feedback_Score": 0.0,
                "Parent_Involvement": "Unknown",
                "Demographic_Group": "Unknown",
                "Internet_Access": "Unknown",
                "Learning_Disabilities": "Unknown",
                "Preferred_Learning_Style": "Unknown",
                "Language_Proficiency": "Unknown",
                "Participation_Rate": "Unknown",
                "Completion_Time_Days": 0,
                "Performance_Score": 0.0,
                "Course_Completion_Rate": 0.0,
                "Completion_Date": '12/31/9999'
})

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 12, Finished, Available, Finished)

In [64]:
df_filled.printSchema()

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 13, Finished, Available, Finished)

root
 |-- Student_ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = false)
 |-- Gender: string (nullable = false)
 |-- Grade_Level: string (nullable = true)
 |-- Course_ID: string (nullable = true)
 |-- Course_Name: string (nullable = true)
 |-- Enrollment_Date: string (nullable = true)
 |-- Completion_Date: string (nullable = false)
 |-- Status: string (nullable = false)
 |-- Final_Grade: string (nullable = false)
 |-- Attendance_Rate: double (nullable = false)
 |-- Time_Spent_on_Course_hrs: double (nullable = false)
 |-- Assignments_Completed: integer (nullable = false)
 |-- Quizzes_Completed: integer (nullable = false)
 |-- Forum_Posts: integer (nullable = false)
 |-- Messages_Sent: integer (nullable = false)
 |-- Quiz_Average_Score: double (nullable = false)
 |-- Assignment_Scores: string (nullable = true)
 |-- Assignment_Average_Score: double (nullable = false)
 |-- Project_Score: double (nullable = false)
 |-- Extra_Credit: double (nul

### Standerdize Date Formats

In [65]:
df_format = df_filled.withColumn('Enrollment_Date',to_date(col('Enrollment_Date'),'M/d/yyyy')) \
                    .withColumn('Completion_Date',to_date(col('Completion_Date'),'M/d/yyyy'))
display(df_format)

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f22d247b-d455-4e38-a802-b30bf3d7fb44)

### Check For Logical Consistency


In [66]:
df_consistent = df_format.filter(col('Completion_Date') >= col('Enrollment_Date'))

display(df_consistent)

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 116f0b73-cb09-427b-bbef-b57abd9ac70d)

### Transformations


#### Logic = Completion_Time_days = Completion_date - Enrollment_Date


In [67]:
from pyspark.sql.functions import * 

df_days = df_consistent.withColumn('Completion_Time_Days',(col('Completion_Date') - col('Enrollment_Date')).cast('int'))

display(df_days)

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e51f08a2-a28c-4aaa-a2e4-a3cac8d6413f)


##### Logic (Quiz_average_score *0.2) + (Assigmnet_average_score*0.2) + (Project_score *0.2)

In [68]:
df_score = df_days.withColumn('Performance_Score',((col('Quiz_Average_Score') * 0.2) + (col('Assignment_Average_Score') * 0.2) + (col('Project_Score') * 0.2)))

display(df_score)

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d6a70f2d-7904-4ee7-8470-385f44749390)

## Course_Completion_Rate

In [69]:
df_completion = df_score.withColumn('Course_Completion_Rate', when(col('Completion_Time_Days') <= 90,'On Time').otherwise('Delayed'))

display(df_completion)

StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c262caf8-2fbc-4e46-8a12-7cdedc9ac2c6)

In [70]:
df_completion.printSchema()

StatementMeta(, fa045363-e7fa-4500-9c93-ffa51a24b280, 72, Finished, Available, Finished)

root
 |-- Student_ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = false)
 |-- Gender: string (nullable = false)
 |-- Grade_Level: string (nullable = true)
 |-- Course_ID: string (nullable = true)
 |-- Course_Name: string (nullable = true)
 |-- Enrollment_Date: date (nullable = true)
 |-- Completion_Date: date (nullable = true)
 |-- Status: string (nullable = false)
 |-- Final_Grade: string (nullable = false)
 |-- Attendance_Rate: double (nullable = false)
 |-- Time_Spent_on_Course_hrs: double (nullable = false)
 |-- Assignments_Completed: integer (nullable = false)
 |-- Quizzes_Completed: integer (nullable = false)
 |-- Forum_Posts: integer (nullable = false)
 |-- Messages_Sent: integer (nullable = false)
 |-- Quiz_Average_Score: double (nullable = false)
 |-- Assignment_Scores: string (nullable = true)
 |-- Assignment_Average_Score: double (nullable = false)
 |-- Project_Score: double (nullable = false)
 |-- Extra_Credit: double (nullable

In [71]:
df_completion.createOrReplaceTempView('new_data')

StatementMeta(, fa045363-e7fa-4500-9c93-ffa51a24b280, 73, Finished, Available, Finished)

In [72]:
display(spark.sql('select * from new_data'))

StatementMeta(, fa045363-e7fa-4500-9c93-ffa51a24b280, 74, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 193f3d09-4c81-40a3-b5c0-4e569a1141e1)

In [74]:
fabric_silver_path = f"abfss://New_Project@onelake.dfs.fabric.microsoft.com/LH_Silver.Lakehouse/Tables/silver_data"

try :
    print('Reading data from teh silver table')
    spark.read.format('delta').load(fabric_silver_path).createOrReplaceTempView('silver_data')

except:
    print('No Table foud,Creating a silver table:')
    create_table = f"""create table silver_data(
            Student_ID string,
            Name string,
            Age int not null,
            Gender string not null,
            Grade_Level string,
            Course_ID string,
            Course_Name string,
            Enrollment_Date date,
            Completion_Date Date,
            Status string not null,
            Final_Grade string not null,
            Attendance_Rate double not null,
            Time_Spent_on_Course_hrs double not null,
            Assignments_Completed integer not null,
            Quizzes_Completed integer not null,
            Forum_Posts integer not null,
            Messages_Sent integer not null,
            Quiz_Average_Score string,
            Assignment_Scores string,
            Assignment_Average_Score double,
            Project_Score double not null,
            Extra_Credit double not null,
            Overall_Performance double not null,
            Feedback_Score double not null,
            Parent_Involvement string not null,
            Demographic_Group string not null,
            Internet_Access string not null,
            Learning_Disabilities string not null,
            Preferred_Learning_Style string not null,
            Language_Proficiency string not null,
            Participation_Rate string not null,
            Completion_Time_Days int,
            Performance_Score double not null,
            Course_Completion_Rate string not null,
            Processing_Date date );"""

    spark.sql(create_table)
    spark.read.format('delta').load(fabric_silver_path).createOrReplaceTempView('silver_data')


StatementMeta(, 5d5840f5-fc1d-4644-9f53-d17b95684fcc, 22, Finished, Available, Finished)

Reading data from teh silver table


In [75]:
sql_statement = f"""merge into silver_data as target
                    using new_data as source
                    on target.Student_ID = source.Student_ID and target.Course_ID = source.Course_Id

                    when matched then
                    update set 
                            target.Name = source.Name,
                            target.Age = source.Age,
                            target.Gender = source.Gender,
                            target.Grade_Level = source.Grade_Level,
                            target.Course_Name = source.Course_Name,
                            target.Enrollment_Date = source.Enrollment_Date,
                            target.Completion_Date = source.Completion_Date,
                            target.Status = source.Status,
                            target.Final_Grade = source.Final_Grade,
                            target.Attendance_Rate = source.Attendance_Rate,
                            target.Time_Spent_on_Course_hrs = source.Time_Spent_on_Course_hrs,
                            target.Assignments_Completed = source.Assignments_Completed,
                            target.Quizzes_Completed = source.Quizzes_Completed,
                            target.Forum_Posts = source.Forum_Posts,
                            target.Messages_Sent = source.Messages_Sent,
                            target.Quiz_Average_Score = source.Quiz_Average_Score,
                            target.Assignment_Scores = source.Assignment_Scores,
                            target.Assignment_Average_Score = source.Assignment_Average_Score,
                            target.Project_Score = source.Project_Score,
                            target.Extra_Credit = source.Extra_Credit,
                            target.Overall_Performance = source.Overall_Performance,
                            target.Feedback_Score = source.Feedback_Score,
                            target.Parent_Involvement = source.Parent_Involvement,
                            target.Demographic_Group = source.Demographic_Group,
                            target.Internet_Access = source.Internet_Access,
                            target.Learning_Disabilities = source.Learning_Disabilities,
                            target.Preferred_Learning_Style = source.Preferred_Learning_Style,
                            target.Language_Proficiency = source.Language_Proficiency,
                            target.Participation_Rate = source.Participation_Rate,
                            target.Completion_Time_Days = source.Completion_Time_Days,
                            target.Performance_Score = source.Performance_Score,
                            target.Course_Completion_Rate = source.Course_Completion_Rate,
                            target.Processing_Date = source.Processing_Date
                    
                    when not matched then
                    insert (Student_ID, Name, Age, Gender, Grade_Level, Course_ID, Course_Name, Enrollment_Date, Completion_Date,
                                Status, Final_Grade, Attendance_Rate, Time_Spent_on_Course_hrs, Assignments_Completed, Quizzes_Completed,
                                Forum_Posts, Messages_Sent, Quiz_Average_Score, Assignment_Scores, Assignment_Average_Score, Project_Score,
                                Extra_Credit, Overall_Performance, Feedback_Score, Parent_Involvement, Demographic_Group, Internet_Access,
                                Learning_Disabilities, Preferred_Learning_Style, Language_Proficiency, Participation_Rate, Completion_Time_Days,
                                Performance_Score, Course_Completion_Rate, Processing_Date) 
                    values(source.Student_ID, source.Name, source.Age, source.Gender, source.Grade_Level, source.Course_ID, source.Course_Name, 
                                source.Enrollment_Date, source.Completion_Date, source.Status, source.Final_Grade, source.Attendance_Rate, 
                                source.Time_Spent_on_Course_hrs, source.Assignments_Completed, source.Quizzes_Completed, source.Forum_Posts, 
                                source.Messages_Sent, source.Quiz_Average_Score, source.Assignment_Scores, source.Assignment_Average_Score, 
                                source.Project_Score, source.Extra_Credit, source.Overall_Performance, source.Feedback_Score, source.Parent_Involvement, 
                                source.Demographic_Group, source.Internet_Access, source.Learning_Disabilities, source.Preferred_Learning_Style, 
                                source.Language_Proficiency, source.Participation_Rate, source.Completion_Time_Days, source.Performance_Score, 
                                source.Course_Completion_Rate,source.Processing_Date)"""

spark.sql(sql_statement).show()

StatementMeta(, fa045363-e7fa-4500-9c93-ffa51a24b280, 77, Finished, Available, Finished)

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                8|               0|               0|                8|
+-----------------+----------------+----------------+-----------------+



In [76]:
df = spark.sql("SELECT * FROM LH_Silver.silver_data LIMIT 1000")
display(df)

StatementMeta(, fa045363-e7fa-4500-9c93-ffa51a24b280, 78, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a3ce0c01-2e97-4d43-974a-92451433d5db)